In [2]:
import pandas as pd
import sys
import pyreadstat
import glob
import tqdm

In [51]:
files = glob.glob("*.xlsx")
files

['R01ED CDI Bilingual 4 Years, Master.xlsx',
 'R01ED CDI Bilingual 3.5 Years, Master.xlsx',
 'HDPM CDI Monolingual 22 month.xlsx',
 'R01ED CDI Monolingual 4 Years, Master.xlsx',
 'HDPM IDHC Bilingual 25 month.xlsx',
 'R01ED CDI Monolingual 2.5 Years, Master.xlsx',
 'R01ED CDI Bilingual 2.5 Years, Master.xlsx',
 'R01ED IDHC Bilingual 4 Years, Master.xlsx',
 'HDPM IDHC Bilingual 22 month.xlsx',
 'HDPM CDI Bilingual 30 month.xlsx',
 'HDPM CDI Monolingual 25 month.xlsx',
 'R01ED CDI Monolingual 3.5 Years, Master.xlsx',
 'R01ED CDI Bilingual 3 Years, Master.xlsx',
 'R01ED IDHC Bilingual 2.5 Years, Master.xlsx',
 '~$HDPM CDI Monolingual 22 month.xlsx',
 'HDPM IDHC Bilingual 30 month.xlsx',
 'HDPM CDI Bilingual 22 month.xlsx',
 'R01ED CDI Monolingual 3 Years, Master.xlsx',
 'R01ED IDHC Bilingual 3.5 Years, Master.xlsx',
 'R01ED IDHC Bilingual 3 Years, Master.xlsx',
 '~$HDPM CDI Bilingual 22 month.xlsx',
 'HDPM CDI Bilingual 25 month.xlsx',
 'HDPM CDI Monolingual 30 month.xlsx']

In [28]:
## for the instruction file, use one of the files

df_dict = pd.read_excel(files[0], sheet_name=None, header=1)
all_df = []
for name,df in df_dict.items():
    if "Not in Use" in name:
        continue
    df = df.dropna(axis=1, how="all").fillna("")
    division_line_index = df[df["Testing Code"]==''].first_valid_index()
    df = df[:division_line_index]
    df["type"] = name
    all_df.append(df)
    print(df)

all_df = pd.concat(all_df)
    

      Testing Code HDPM001 HDPM002 HDPM003 HDPM004 HDPM005 HDPM006 HDPM007  \
0          baa baa     1.0     1.0     1.0     0.0     0.0     1.0     0.0   
1        choo choo     0.0     1.0     1.0     1.0     1.0     0.0     0.0   
2   cockadoodledoo     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3              grr     0.0     0.0     1.0     1.0     0.0     0.0     1.0   
4             meow     0.0     1.0     1.0     1.0     1.0     0.0     0.0   
5              moo     0.0     1.0     1.0     1.0     1.0     1.0     0.0   
6             ouch     1.0     1.0     1.0     1.0     1.0     0.0     0.0   
7      quack quack     0.0     1.0     1.0     1.0     1.0     0.0     0.0   
8            uh oh     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
9            vroom     0.0     0.0     1.0     1.0     0.0     0.0     0.0   
10       woof woof     0.0     1.0     1.0     1.0     1.0     0.0     0.0   
11         yum yum     1.0     0.0             1.0     0.0     1

In [35]:
l1 = all_df["Testing Code"].to_list() 
l2 = all_df["type"].to_list()

import numpy as np 
instruction = pd.DataFrame(np.array([l1,l2]).transpose(),columns=["item","type"])

In [36]:
instruction

,item,type
0,baa baa,Sound Effects and Animal Sounds
1,choo choo,Sound Effects and Animal Sounds
2,cockadoodledoo,Sound Effects and Animal Sounds
3,grr,Sound Effects and Animal Sounds
4,meow,Sound Effects and Animal Sounds
...,...,...
565,shoeses,C. Word Endings Part 2
566,sockses,C. Word Endings Part 2
567,teeths,C. Word Endings Part 2
568,toeses,C. Word Endings Part 2


In [42]:
choices = ["produces/does not produce"]*len(instruction)
instruction["choices"] = choices
instruction["itemID"] = ["item_"+str(i) for i in list(range(1,len(instruction["item"])+1))]
instruction["complexity_category"] = ['']*len(instruction["item"])
instruction["gloss"] = instruction["item"]

In [43]:
from collections import Counter
items = list(instruction.item)
uniq = []
for i in items:
    if i not in uniq:
        uniq.append(i)
    else:
        uniq.append(i+"_2")
instruction.item = uniq

In [44]:
len(set(instruction.item))

570

In [45]:
instruction.to_csv("Spanish_English.csv", index=False)

In [46]:
instruction

,item,type,choices,itemID,complexity_category,gloss
0,baa baa,Sound Effects and Animal Sounds,produces/does not produce,item_1,,baa baa
1,choo choo,Sound Effects and Animal Sounds,produces/does not produce,item_2,,choo choo
2,cockadoodledoo,Sound Effects and Animal Sounds,produces/does not produce,item_3,,cockadoodledoo
3,grr,Sound Effects and Animal Sounds,produces/does not produce,item_4,,grr
4,meow,Sound Effects and Animal Sounds,produces/does not produce,item_5,,meow
...,...,...,...,...,...,...
565,shoeses,C. Word Endings Part 2,produces/does not produce,item_566,,shoeses
566,sockses,C. Word Endings Part 2,produces/does not produce,item_567,,sockses
567,teeths,C. Word Endings Part 2,produces/does not produce,item_568,,teeths
568,toeses,C. Word Endings Part 2,produces/does not produce,item_569,,toeses


In [54]:
## now loop through all kids to make the data files
all_kids = []
columns = instruction.itemID.to_list()

for file in files:
    child_name = file.split(".")[0]
    df_dict = pd.read_excel(files[0], sheet_name=None, header=1)
    all_df = []
    for name,df in df_dict.items():
        if "Not in Use" in name:
            continue
        df = df.dropna(axis=1, how="all").fillna("")
        division_line_index = df[df["Testing Code"]==''].first_valid_index()
        df = df[:division_line_index]
        all_df.append(df)
    all_df = pd.concat(all_df)
    df = all_df.transpose()
    df.columns = columns
    df["Child"]=child_name
    df = df.drop("Testing Code",axis=0)
    df = df.reset_index()
    all_kids.append(df)
    
all_kids = pd.concat(all_kids)
all_kids
    

,index,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,...,item_562,item_563,item_564,item_565,item_566,item_567,item_568,item_569,item_570,Child
0,R01ED400,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,"R01ED CDI Bilingual 4 Years, Master"
1,R01ED402,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,1,0,1,"R01ED CDI Bilingual 4 Years, Master"
2,R01ED403,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,1,1,0,0,0,1,0,0,"R01ED CDI Bilingual 4 Years, Master"
3,R01ED408,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,"R01ED CDI Bilingual 4 Years, Master"
4,R01ED409,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,"R01ED CDI Bilingual 4 Years, Master"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,R2130Mo139,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,1,0,0,0,0,0,0,HDPM CDI Monolingual 30 month
90,R2130Mo141,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,1,0,0,HDPM CDI Monolingual 30 month
91,R2130Mo143,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,HDPM CDI Monolingual 30 month
92,R2130Mo144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,HDPM CDI Monolingual 30 month


In [59]:
columns = list(all_kids.columns)
columns.remove("index")
columns.insert(0,"session")

In [60]:
all_kids.columns = columns

In [61]:
all_kids

,session,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,...,item_562,item_563,item_564,item_565,item_566,item_567,item_568,item_569,item_570,Child
0,R01ED400,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,"R01ED CDI Bilingual 4 Years, Master"
1,R01ED402,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,1,0,1,"R01ED CDI Bilingual 4 Years, Master"
2,R01ED403,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,1,1,0,0,0,1,0,0,"R01ED CDI Bilingual 4 Years, Master"
3,R01ED408,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,"R01ED CDI Bilingual 4 Years, Master"
4,R01ED409,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,"R01ED CDI Bilingual 4 Years, Master"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,R2130Mo139,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,1,0,0,0,0,0,0,HDPM CDI Monolingual 30 month
90,R2130Mo141,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,1,0,0,HDPM CDI Monolingual 30 month
91,R2130Mo143,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,HDPM CDI Monolingual 30 month
92,R2130Mo144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,HDPM CDI Monolingual 30 month


In [64]:
first_column = all_kids.pop('Child')
all_kids.insert(0, 'Child', first_column)

In [66]:
all_kids.to_csv("Spanish_English_data(individual information missing).csv",index=False)